In [12]:
import pandas as pd
import socket
import time

# MODIFIED CLASS: Uses standard UDP to guarantee Wireshark capture
# without needing Admin/Root password.
class RawTcpTransport:
    def __init__(self, src_ip, dst_ip, src_port, dst_port):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
        
        # Use UDP (SOCK_DGRAM) because it doesn't need a "Handshake"
        # It just blasts packets, which is perfect for this simulation.
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    def send(self, data, flags=0x02):
        # We send the data to the destination port (12345)
        # Wireshark will see this as a UDP packet.
        try:
            self.sock.sendto(data, (self.dst_ip, 12345))
            print(f"--> SENT REAL PACKET: {data}")
        except Exception as e:
            print(f" Error sending: {e}")

# Initialize the transporter
# sending to localhost (127.0.0.1)
transport = RawTcpTransport('127.0.0.1', '127.0.0.1', 12345, 80)
print(" Standard Socket Setup Complete. (Wireshark will see this!)")

 Standard Socket Setup Complete. (Wireshark will see this!)


In [13]:
# 1. Load the CSV file you saved in the 'networksProject' folder
filename = "group05_chat_input.csv" 

try:
    messages_df = pd.read_csv(filename)
    print(f"Loaded {len(messages_df)} messages from file.")
    
    # 2. Loop through the file and send each message
    print("\n--- STARTING TRANSMISSION ---\n")
    
    for index, row in messages_df.iterrows():
        # Create the message string
        msg_payload = str(row['message'])
        src = row['src_app']
        dst = row['dst_app']
        
        # Combine them into one text packet
        full_payload = f"[{src}->{dst}] {msg_payload}"
        
        # Send it!
        transport.send(full_payload.encode('utf-8'))
        
        # Wait a bit so it looks like real conversation
        time.sleep(0.5)

    print("\n--- TRANSMISSION COMPLETE ---")

except FileNotFoundError:
    print("ERROR: Could not find 'group05_chat_input.csv'.")
    print("Make sure you saved the TextEdit file in the SAME folder as this notebook!")

Loaded 6 messages from file.

--- STARTING TRANSMISSION ---

--> SENT REAL PACKET: b'[client_A->server] HELLO_SERVER'
--> SENT REAL PACKET: b'[client_A->server] CONNECT client_B'
--> SENT REAL PACKET: b'[server->client_A] CONNECTED_TO client_B'
--> SENT REAL PACKET: b'[client_A->client_B] How are you?'
--> SENT REAL PACKET: b'[client_B->client_A] I am good thanks'
--> SENT REAL PACKET: b'[client_A->client_B] Bye!'

--- TRANSMISSION COMPLETE ---
